<a href="https://colab.research.google.com/github/alexontour/snippets/blob/main/snip_fhir_create_bulk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author: Alexander Kollmann, 12/2022**

---

**Funktion**

Bulk-Upload aus CSV-Datei (Patient & Observation - HR)

Verwenden der Bibliothek "fhirclient"

---

**Referenzen**

https://pypi.org/project/fhirclient/


In [ ]:
import json
import requests
from collections import OrderedDict
from io import StringIO
from IPython.display import IFrame

In [ ]:
# Base URL zum FHIR-Server
# https://confluence.hl7.org/display/FHIR/Public+Test+Servers

url = "http://hapi.fhir.org/baseR4/"
#url = 'https://server.fire.ly/'
#url = "	http://localhost:8080/fhir"

# Header definieren
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}

In [ ]:
# Zufalls-Wert für Namen generieren um Dupletten zu vermeiden

import random
import string

def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_letters
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

In [ ]:
!pip install fhirclient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 683 kB 5.2 MB/s 
     |████████████████████████████████| 41 kB 325 kB/s 


In [ ]:
import fhirclient.models.patient as p
import fhirclient.models.humanname as hn
import fhirclient.models.contactpoint as cp
import fhirclient.models.fhirdate as fd
import fhirclient.models.identifier as ident
import fhirclient.models.observation as obs
import fhirclient.models.coding as co
import fhirclient.models.codeableconcept as coco
import fhirclient.models.quantity as qua
import fhirclient.models.fhirreference as ref

from fhirclient import client

import pandas as pd
import numpy as np

import time
import itertools

In [ ]:
# import patient-list
# Datei von Google Drive lesen (Wichtig: die Datei muss freigegeben werden)

doc_url='https://drive.google.com/file/d/1K9aeof3TFZYPPOKayZ-oQ_duza96_ZAX/view?usp=sharing'

file_id=doc_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url, sep=';')

df = df.applymap(str)

# data cleaning
df['rec_id']= df['rec_id']
df['sex']= df['sex'].replace('f', 'female')
df['sex']= df['sex'].replace('m', 'male')

print('Number of records :', len(df))
print(df.head())

Number of records : 2
  rec_id     sex date_of_birth given_name surname
0      1    male    1967-12-07       Alex   Kolli
1      2  female    1959-03-04       Juli   Huber


In [ ]:
# import value-list
# Datei von Google Drive lesen (Wichtig: die Datei muss freigegeben werden)

doc_url='https://drive.google.com/file/d/1KCK-YqM3k_Zjkoa7o0Z30cEo1wXEXPds/view?usp=sharing'

file_id=doc_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df_val = pd.read_csv(dwn_url, sep=',')

df_val = df_val.applymap(str)

# data cleaning
df_val.drop('DistanceMeters', inplace=True, axis=1)
df_val['HeartRateBpm'] = df_val['HeartRateBpm'].astype(float)

print('Number of records :', len(df_val))
print(df_val.head())

Number of records : 615
                       Time  HeartRateBpm
0  2022-11-24T16:51:38.000Z         106.0
1  2022-11-24T16:51:39.000Z         104.0
2  2022-11-24T16:51:40.000Z         105.0
3  2022-11-24T16:51:43.000Z         104.0
4  2022-11-24T16:51:44.000Z         106.0


In [ ]:
# https://hl7.org/fhir/observation-example-heart-rate.json.html

def create_observation_fhirclient(patient_id, ts, hr):
  coding = co.Coding()
  coding.system = "https://loinc.org"
  coding.code = "8867-4"
  coding.display = "Heart rate"
  code = coco.CodeableConcept()
  code.coding = [coding]
  code.text = "Heart rate"

  #Create a new observation using fhir.resources, we enter status and code inside the constructor since theuy are necessary to validate an observation
  observation = obs.Observation()
  observation.status = "final"
  observation.code = code
  #Set our category in our observation, category which hold codings which are composed of system, code and display
  coding = co.Coding()
  coding.system = "https://terminology.hl7.org/CodeSystem/observation-category"
  coding.code = "vital-signs"
  coding.display = "Vital Signs"
  category = coco.CodeableConcept()
  category.coding = [coding]
  observation.category = [category]

  #Set our effective date time in our observation
  observation.effectiveDateTime = fd.FHIRDate(ts)

  #Set our valueQuantity in our observation, valueQuantity which is made of a code, a unir, a system and a value
  valueQuantity = qua.Quantity()
  valueQuantity.code = "/min"
  valueQuantity.unit = "beats/minute"
  valueQuantity.system = "https://unitsofmeasure.org"
  valueQuantity.value = hr
  observation.valueQuantity = valueQuantity

  #Setting the reference to our patient using his id
  reference = ref.FHIRReference()
  reference.reference = f"Patient/{patient_id}"
  observation.subject = reference

  #print(json.dumps(observation.as_json(), indent=2))

  io = StringIO()
  json.dump(observation.as_json(), io, indent=2)

  #headers = {'Content-Type': 'application/json'}
  start = time.time()
  response = requests.request("POST", url + "Observation", headers=headers, data=io.getvalue())
  end = time.time()

  print(response)

  obs_id = json.loads(response.text)['id']
  print("Observation-ID: " + obs_id + " " + str(round((end - start), 1)) + " ms")

In [ ]:
# Option 4: FHIR-Ressource über fhirclient erstellen
# Patienten speichern (RAND-im Nachnamen)

rand = get_random_string(5)

limit = 100
for index, row in itertools.islice(df.iterrows(), limit):
  #for index, row in df.iterrows():
    patient = p.Patient() # not using rec_id as pandas id, leaving empty
    patient.gender = row['sex']
    name = hn.HumanName()
    name.given = [row['given_name']]
    name.family = row['surname'] + rand
    name.use = 'official'
    patient.name = [name]
    patient.birthDate = fd.FHIRDate(row['date_of_birth'])
    id = ident.Identifier()
    id.system = 'http://clientregistry.org/openmrs'
    id.value = row['rec_id']
    patient.identifier = [id]
    #print(json.dumps(patient.as_json(), indent=2))

    io = StringIO()
    json.dump(patient.as_json(), io, indent=2)

    #headers = {'Content-Type': 'application/json'}
    start = time.time()
    response = requests.request("POST", url + "Patient", headers=headers, data=io.getvalue())
    end = time.time()
    print(response)
    patient_id = json.loads(response.text)['id']
    print("Patient-ID: " + patient_id + " " + str(round((end - start), 1)) + " ms")
    
    # Observation zum Patienten speichern
    for index, row in df_val.iterrows():
      create_observation_fhirclient(patient_id, row['Time'], row['HeartRateBpm'])

<Response [201]>
Patient-ID: 4c057ebe-d5e1-4e03-923b-c11efc020181 1.4 ms
<Response [201]>
Observation-ID: 44272e63-f30b-41af-a424-f30a0f839e2c 1.0 ms
<Response [201]>
Observation-ID: d8e602c7-916d-4a7e-96ae-18f3eb65d681 0.4 ms
<Response [201]>
Observation-ID: 16e8afb5-18e6-411b-8082-941668939934 0.4 ms
<Response [201]>
Observation-ID: b8450bfd-6811-4906-8fbc-c62ea8156a3b 0.4 ms
<Response [201]>
Observation-ID: 4e3a35c2-be56-4fdb-83d1-1c22244b79f1 0.4 ms
<Response [201]>
Observation-ID: c5e0aa37-1293-4e70-b2af-f7d4de320385 0.4 ms
<Response [201]>
Observation-ID: 44a30718-e62d-4654-a4fe-7d687dedb463 0.4 ms
<Response [201]>
Observation-ID: 36bff06b-c4e7-4ada-8fa7-3cade91ea57a 0.4 ms
<Response [201]>
Observation-ID: b7891aea-a629-4295-99b9-573b85fb0253 0.4 ms
<Response [201]>
Observation-ID: 5e3f1fc2-3eda-4b8c-8dfd-f6f66441d14a 0.4 ms
<Response [201]>
Observation-ID: 5af4d3da-55a7-4c10-82d7-e409b0c4d5da 0.4 ms
<Response [201]>
Observation-ID: 56679236-ed1d-4112-aa20-257f928ca3c7 0.4 ms
<Re